In [1]:
from mindquantum.core.circuit import Circuit, UN
from mindquantum.core.gates import H, ZZ, RX, XX, YY, RY, X
from mindquantum.core.operators import Hamiltonian, QubitOperator
from mindquantum.framework import MQAnsatzOnlyLayer
from mindquantum.simulator import inner_product, Simulator
from mindquantum.algorithm.nisq import HardwareEfficientAnsatz
from mindquantum.core.circuit import shift, Circuit
from mindquantum.core.operators import TimeEvolution, QubitOperator

import matplotlib.pyplot as plt
import networkx as nx
import mindspore.nn as nn
import mindspore as ms
import numpy as np  
import random
import pandas as pd

qubit_num = 10

# 定义问题哈密顿量及问题层线路circuit1

In [2]:
def Construct_Hamiltonian_C():
    H_C = QubitOperator() 
    H_C += QubitOperator('',28.5)+ QubitOperator('Z0', -4.5) + QubitOperator('Z1', -5)+ QubitOperator('Z2', -4)+ QubitOperator('Z3', -2.5)+ QubitOperator('Z4', -3) + QubitOperator('Z5', -2.5)+ QubitOperator('Z6', -2.5)+ QubitOperator('Z7', -0.5)+ QubitOperator('Z8', -1)+ QubitOperator('Z9', -3)
    
    return H_C

In [3]:
H_C = Construct_Hamiltonian_C()
print (H_C)

57/2 [] +
-9/2 [Z0] +
-5 [Z1] +
-4 [Z2] +
-5/2 [Z3] +
-3 [Z4] +
-5/2 [Z5] +
-5/2 [Z6] +
-1/2 [Z7] +
-1 [Z8] +
-3 [Z9] 


In [4]:
hc = H_C - 28.5*QubitOperator(())
print (hc)

-9/2 [Z0] +
-5 [Z1] +
-4 [Z2] +
-5/2 [Z3] +
-3 [Z4] +
-5/2 [Z5] +
-5/2 [Z6] +
-1/2 [Z7] +
-1 [Z8] +
-3 [Z9] 


In [5]:
# circuit1是问题哈密顿量H_C中去掉单位算子I构造的电路，属于问题层(根据目标函数构建)
circuit1 = TimeEvolution(hc,{'gamma':0.5}).circuit
circuit1.svg()

# 构造混合层线路𝑒^(−𝑖𝛽𝐻_𝑀 )

In [6]:
def build_hb(para):              #自定义一个电路构建块hc（目标哈密顿量）， # 
    hb = Circuit()               # 创建量子线路
    
    hb += RX({para:2}).on(0, [5,6,7,8,9])  
    
    hb += RX({para:2}).on(1, [6,9])
    
    hb += RX({para:2}).on(2, [5,9])
    
    hb += RX({para:2}).on(3, [5,6,8,9])
    
    hb += RX({para:2}).on(4, [5,7,8,9])  
    
    hb += RX({para:2}).on(5, [0,2,3,4])
    
    hb += RX({para:2}).on(6, [0,1,3])
    
    hb += RX({para:2}).on(7, [0,4])
    
    hb += RX({para:2}).on(8, [0,3,4])
    
    hb += RX({para:2}).on(9, [0,1,2,3,4])
    

    
    
#    hb.barrier()                    # 添加Barrier以方便展示线路
    return hb

In [7]:
# pylint: disable=W0104
circuit2 = build_hb('beta')
circuit2.svg()
#circuit2.svg().to_file(filename='H_Bcircuit.svg')

# 总线路ansatz

In [8]:
def build_ansatz(p):                    # g是max-cut问题的图，p是ansatz线路的层数                      
    circ = UN(X, qubit_num)             # 制备初始态，创建量子线路
    for i in range(p):
        circ += TimeEvolution(hc, {f'γ{i}':0.5}).circuit       # 问题层H_C  （依目标函数而设计）
        circ += build_hb(f'β{i}')       # 混合层   （依约束条件1而设计）
        
    return circ

In [9]:
p = 1
build_ansatz(p).svg()                                      #展示完整的量子线路
#build_ansatz(p).svg().to_file(filename='Full_circuit.svg')   #展示完整的量子线路

# 单次实验

# _main_文件
主要是为了比较期望值和成功概率随层数p的变化情况

# 以Excel（.xlsx）文件的形式输出训练结果

In [10]:
if __name__ == "__main__":
    #创建一个空的DataFrame来存储结果
    results = []

    for p in range(6, 11):
        print('The p value is: {}'.format(p))
        for t in range(1, 501):
            ham = Hamiltonian(Construct_Hamiltonian_C()) # 生成哈密顿量
            circ = build_ansatz(p)                  # 生成ansatz线路

            ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")
            sim = Simulator('mqvector', circ.n_qubits)                     # 创建模拟器，backend使用‘mqvector’，能模拟5个比特（'circ'线路中包含的比特数）
            grad_ops = sim.get_expectation_with_grad(ham, circ)            # 获取计算变分量子线路的期望值和梯度的算子
            net = MQAnsatzOnlyLayer(grad_ops)                              # 生成待训练的神经网络
            opti = nn.Adam(net.trainable_params(), learning_rate = 0.05)     # 设置针对网络中所有可训练参数、学习率为0.05的Adam优化器
            train_net = nn.TrainOneStepCell(net, opti)                     # 对神经网络进行一步训练                   
                
            
            for i in range(500):
                energy = train_net()
                #if i%500 == 0:
                #    print('i = %d energy = %f'%(i,energy))
                
            pr = dict(zip(circ.params_name, net.weight.asnumpy())) 
            state = circ.get_qs(pr=pr)
            
            v = [0]* 2**qubit_num
            v[992] = 1
            fid = np.abs(np.vdot(v,state))**2 

            # 将结果添加到列表
            results.append({"p": p, "t": t, "energy": energy, "fidelity": fid})
            
            #命令交互行输出，便于区分（实际上没啥必要）            
            #print('t: {}'.format(t))

        # 将列表转换为DataFrame
        df = pd.DataFrame(results)

        # 将DataFrame写入Excel文件
        df.to_excel(r"D:\pyBUPTAIML\mywork\远程服务器备份\liguanghui（1paper)\科研\MWVC\MWVC paper\MWVC_QAOA+_v2\实验4\code\箱型图code\bipartite graph code\BG107.xlsx", index=False)
    

The p value is: 6
The p value is: 7
The p value is: 8
The p value is: 9
The p value is: 10
